<a href="https://colab.research.google.com/github/Bharath-tars/House_prices_Supervised_model/blob/main/House_me.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importing all the essential libraries required for solving the problem statement

In [ ]:
import copy, math
import numpy as np
import matplotlib.pyplot as plt

Importing the dataset from the filepath and dividing it into features and target values

**point to remember**
->The filepath of the dataset should we same as source code file otherwise change the file path according to the directories.

In [ ]:
data = np.loadtxt("vertopal.com_houses (1).ipynb", delimiter=',')
X_train = data[:,:4]
y_train = data[:,4:]
X_features = ['size(sqft)','bedrooms','floors','age']

Plotting the dataset values inform of graphs to get an precise look into the analysis

In [ ]:
fig,ax=plt.subplots(1, 4, figsize=(12, 3), sharey=True)
for i in range(len(ax)):
    ax[i].scatter(X_train[:,i],y_train)
    ax[i].set_xlabel(X_features[i])
ax[0].set_ylabel("Price (1000's)")
plt.show()

**Optional** -- This is just for just knowing the info about the training set.

In [ ]:
print(f"X Shape: {X_train.shape}, X Type:{type(X_train)})")
print(X_train)
print(f"y Shape: {y_train.shape}, y Type:{type(y_train)})")
print(y_train)

Z-normalization function which normalizes the features into readable and easy calculated format.. it includes numpy functions like mean and standard deviation

In [ ]:
def zscore_normalize_features(X):
    mu     = np.mean(X, axis=0)                 
    sigma  = np.std(X, axis=0)                  
    X_norm = (X - mu) / sigma      
    return (X_norm, mu, sigma)

This is the initialization of the zscore function 

In [ ]:
X_norm, X_mu, X_sigma = zscore_normalize_features(X_train)
print(f"X_mu = {X_mu}, \nX_sigma = {X_sigma}")
print(f"Peak to Peak range by column in Raw        X:{np.ptp(X_train,axis=0)}")   
print(f"Peak to Peak range by column in Normalized X:{np.ptp(X_norm,axis=0)}")

Basically when we are working with supervised learning algorithms especially linear regresion we use bias(b) and coeffcient(w) as parameters to make our ml model more powerful

In [ ]:
b_init = 785.1811367994083
w_init = np.array([ 0.39133535, 18.75376741, -53.36032453, -26.42131618])
print(f"w_init shape: {w_init.shape}, b_init type: {type(b_init)}")

This is the linear regression model function

In [ ]:
def predict(x, w, b): 
    p = np.dot(x, w) + b     
    return p    

Shape of the training set

In [ ]:
x_vec = X_train[0,:]
print(f"x_vec shape {x_vec.shape}, x_vec value: {x_vec}")

f_wb = predict(x_vec,w_init, b_init)
print(f"f_wb shape {f_wb.shape}, prediction: {f_wb}")

This is the cost function which is gonna return the differnece between the target and prediction which is ultimate factor which determines the quality and performance of our algorithm

In [ ]:
def compute_cost(X, y, w, b): 
    m = X.shape[0]
    cost = 0.0
    for i in range(m):                                
        f_wb_i = np.dot(X[i], w) + b           
        cost = cost + (f_wb_i - y[i])**2       
    cost = cost / (2 * m)                       
    return cost

Initialization of the cost function

In [ ]:
cost = compute_cost(X_train, y_train, w_init, b_init)
print(f'Cost at optimal w : {cost}')

This is the function that returns the partial derivatives of both the parameters that are gonna be used in gradient desent for predicting the values of w and b

In [ ]:
def compute_gradient(X, y, w, b): 
    
    m,n = X.shape           #(number of examples, number of features)
    dj_dw = np.zeros((n,))
    dj_db = 0.

    for i in range(m):                             
        err = (np.dot(X[i], w) + b) - y[i]   
        for j in range(n):                         
            dj_dw[j] = dj_dw[j] + err * X[i, j]    
        dj_db = dj_db + err                        
    dj_dw = dj_dw / m                                
    dj_db = dj_db / m                                
        
    return dj_db, dj_dw

Initialization of partial derivatives

In [ ]:
tmp_dj_db, tmp_dj_dw = compute_gradient(X_train, y_train, w_init, b_init)
print(f'dj_db at initial w,b: {tmp_dj_db}')
print(f'dj_dw at initial w,b: \n {tmp_dj_dw}')

This is the gradient desent that is automatically compute the values of w and b which is used for minimizing the value of cost function

In [ ]:
def gradient_descent(X, y, w_in, b_in, cost_function, gradient_function, alpha, num_iters): 
    
    J_history = []
    w = copy.deepcopy(w_in)  #avoids modifying
    b = b_in
    
    for i in range(num_iters):

        dj_db,dj_dw = gradient_function(X, y, w, b)   
        
        w = w - alpha * dj_dw               
        b = b - alpha * dj_db               
      
        # Saves cost J at each iteration
        if i<100000:      # prevent resource exhaustion 
            J_history.append( cost_function(X, y, w, b))

        # # Prints cost every at intervals 10 times or as many iterations if < 10
        # if i% math.ceil(num_iters / 10) == 0:
        #     print(f"Iteration {i:4d}: Cost {J_history[-1]:8.2f}   ")
        
    return w, b, J_history #returns final w,b and J history for graphing

Initialization of the gradient desent

In [ ]:
initial_w = np.zeros_like(w_init)
initial_b = 0.
w,b,h = gradient_descent(X_train, y_train, initial_w, initial_b,compute_cost, compute_gradient, alpha = 1e-7,num_iters=10)
m,n = X_train.shape
sum=0
for i in range(m):
    sum = (np.dot(X_train[i], w) + b)
    print(f"prediction: {sum}, target value: {y_train[i]}")    

Initialization of the Normalized training set for computing the gradient desent

In [ ]:
w,b,h = gradient_descent(X_norm, y_train, initial_w, initial_b,compute_cost, compute_gradient, alpha = 1.0e-1,num_iters=1000)

Final verification after the training

In [ ]:
m = X_norm.shape[0]
yp = np.zeros(m)
for i in range(m):
    yp[i] = np.dot(X_norm[i], w) + b
    print(yp[i])

Prediction using the trained model

In [ ]:
x_house = np.array([1200, 3, 1, 40])
x_house_norm = (x_house - X_mu) / X_sigma
print(x_house_norm)
x_house_predict = np.dot(x_house_norm, w) + b
print(f" predicted price of a house with 1200 sqft, 3 bedrooms, 1 floor, 40 years old = ${x_house_predict*1000}")

**THE END Peace out::** 
